## Settings

In [1]:
# Importing libraries 

from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
import requests
from requests_html import HTML
from urllib.parse import urlparse
import os
import pandas as pd
from selenium import webdriver
import time
import requests
from urllib.parse import urlparse
import os

In [2]:
# Setting work directories and files: getting current directory
cwd = os.getcwd()
print(cwd)

/Users/dumoura/Dev/PDev/CGScholar/Robot_cit/Notebook


In [3]:
# Setting work directories and files: crating work directories
# Base_dir

BASE_DIR = os.path.dirname(cwd) # base directory

TABLES_DIR = os.path.join(BASE_DIR, "tables")
META_DIR = os.path.join(BASE_DIR, "metada") 

# Make_dirs
os.makedirs(TABLES_DIR, exist_ok=True)
os.makedirs(META_DIR, exist_ok=True)

In [4]:
os.listdir(TABLES_DIR)

['.DS_Store',
 'WorksAuthorIntroRef.csv',
 'Works_40_50_AuthorIntroRef.csv',
 'EPOL481.csv',
 '.ipynb_checkpoints']

## Tables and URLs 

In [ ]:
table_name = input(f"Select a csv document:\n{os.listdir(TABLES_DIR)}\n\n")

In [ ]:
tab_df = pd.read_csv(f"{TABLES_DIR}/{table_name}") #ADD doctitle.csv - doc with the works url
tab_df.columns

In [ ]:
# get URLs and apply to the "work_url_obj" variable/object
work_url_obj = tab_df['Work URL']
# work_url_obj

## Scraping Functions: authors_name, introduction, references

### authors_name ### 

In [ ]:
# function to get the author's name

def authors_name_getter(soup):
    authors_lookup = soup.find_all("li", attrs={"authors publisher"})
    author = authors_lookup[0].text
    author_name = author.split(": ")[1]
#     print(author_name)

    return author_name

### introduction

In [ ]:

def introduction_text_getter(soup):
    try:
        results = soup.find_all("div", attrs={"class":"element section"})
        results_intro = results[0].contents[1]
#         print(results_intro.text)
        return results_intro.text
    
    except IndexError as e:
#         print("Error has occured: ", e)
        return e
    
    pass

### references ### 

In [ ]:
def references_getter(soup):
    try:
        results = soup.find_all("div", attrs={"class":"element section"})
        return results[-1].contents[1].text
    
    except IndexError as e:
#         print("Error has occured: ", e)
        return e
    
    pass

## Crawler ##

### cgscholar login ###

In [ ]:
# ADD your USERNAME and PASSWORD below 

USERNAME = "" 
PASSWORD =  ""
drive_adress = ""
url = "https://cgscholar.com/identity/users/sign_in"

### webdriver ###

In [ ]:
browser = webdriver.Chrome(drive_adress)
url_obj = f"{url}"
browser.get(url_obj)

time.sleep(1.5) 
username_el = browser.find_element_by_name("user[login]")
username_el.send_keys(f"{USERNAME}")

password_el = browser.find_element_by_name("user[password]")
password_el.send_keys(f"{PASSWORD}")

time.sleep(1.5)
submit_btn_el = browser.find_element_by_css_selector("input[type='submit']")
submit_btn_el.click()

### crawling ###

In [ ]:
url_obj = work_url_obj
work = []

for url in url_obj:
    time.sleep(1.5)
    url_docment = str(url)
    
    browser.get(url_docment)
    body_el = browser.find_element_by_css_selector("body")
    html_str = body_el.get_attribute("innerHTML")
    soup = BeautifulSoup(html_str, "lxml")
    
    for i in soup: 
        author_name = authors_name_getter(soup)
        introduction = introduction_text_getter(soup)
        references = references_getter(soup)
        print(author_name)

        work.append([author_name, introduction, references])
        df = pd.DataFrame(work, columns= ["author","introduction", "references"])

        df.to_csv(f"{TABLES_DIR}/Works_40_50_AuthorIntroRef.csv", index=False)

#### Open Work dataset ###

In [131]:
table_name = input(f"Select a csv document:\n{os.listdir(TABLES_DIR)}\n\n")

Select a csv document:
['.DS_Store', 'WorksAuthorIntroRef.csv', 'Works_40_50_AuthorIntroRef.csv', 'EPOL481.csv', '.ipynb_checkpoints']

WorksAuthorIntroRef.csv


In [132]:
tab_df = pd.read_csv(f"{TABLES_DIR}/{table_name}") #ADD doctitle.csv - doc with the works url

In [121]:
# tab_df.head(5)
# print(tab_df.introduction)

### Pre processing - basic cleaning

In [138]:
def cleaning_intro(df):
    intro_texts = str(df.replace('\xa0', ' '))
    intro_texts = str(intro_texts.replace('\n', ' '))
    
    return intro_texts


In [139]:
intro_cld = []

for i in tab_df.introduction:
    intros_ = cleaning_intro(i)
    intro_cld.append(intros_)

tab_df["works_intro"] = intro_cld


In [151]:
tab_df.head(1)

,author,introduction,references,works_intro
0,Aaron Gonzales Aaron,The social context of education aims to identi...,"*Barwegen, L., Falciani, N., Putnam, S., Reame...",The social context of education aims to identi...


In [152]:
tab_df.drop("introduction", axis=1, inplace=True)

In [251]:
df_erros = tab_df["works_intro"] == "list index out of range"
df_intro_texts = tab_df[~df_erros]
df_intro_texts

,author,references,works_intro
0,Aaron Gonzales Aaron,"*Barwegen, L., Falciani, N., Putnam, S., Reame...",The social context of education aims to identi...
1,Abasin Ludin,Those of us working in the educational field h...,Formal and Informal learning are an integral p...
2,Akash Saini,"Blazauskas, T., Maskeliunas, R., Bartkute, R.,...",The education sector has historically been a l...
5,Amena Saleem,"Anthony, Glenda. “Active learning in a constru...","As a second/third grade teacher, I have recent..."
6,Amy Petrinos,"In conclusion, formative assessment can help t...",Image: eSchool News: Teachers share formative ...
7,Amy Trietiak,With a need to meet industry demands for skill...,As both a full-time student and an educator in...
8,Andrew Walker III,Due to the rapid change of technology used in ...,"I consider myself an active video gamer, one o..."
10,Arikpo Dada,Although there is still much research needed i...,“What a year!” That was what many of us said a...
11,Ashley Roberts,Further research is needed to assess the influ...,Figure 1. UNICEF 2018 When accepting a positio...
12,Beth Rock,"*Connectivism Learning Theory. (2021, May 27)....",Researchers and educators focus much of their ...


### Pre processing - creating DFs ###

In [244]:
table_name = input(f"Select a csv document:\n{os.listdir(TABLES_DIR)}\n\n")

Select a csv document:
['.DS_Store', 'WorksAuthorIntroRef.csv', 'Works_40_50_AuthorIntroRef.csv', 'EPOL481.csv', '.ipynb_checkpoints']

EPOL481.csv


In [245]:
df_grade = pd.read_csv(f"{TABLES_DIR}/{table_name}")

In [246]:
df_grade.columns

Index(['Unnamed: 0', 'Expert Reviewer', 'Peer Reviewers',
       'Experience_Peer_Avg', 'Experience_Expert_Avg', 'Experience_Difference',
       'Experience_Rodrigo', 'Experience_Shang', 'Experience_Jen',
       'Referencing_Peer_Avg', 'Referencing_Expert_Avg',
       'Referencing_Difference', 'Referencing_Rodrigo', 'Referencing_Shang',
       'Referencing_Jen', 'Work URL'],
      dtype='object')

In [255]:
dfExperiences = df_grade[['Unnamed: 0', 'Experience_Rodrigo', 'Experience_Shang', 
          'Experience_Jen', 'Peer Reviewers']]

dfExperiences = dfExperiences.rename(columns = {'Unnamed: 0':'author'})

In [256]:
dfExperiences["Experience"]  = df_intro_texts["works_intro"]

In [261]:
df = dfExperiences.dropna()

In [270]:
df = df[['author', "Experience", 'Experience_Rodrigo', 'Experience_Shang', 
          'Experience_Jen', 'Peer Reviewers']]

df.to_csv(f"{TABLES_DIR}/Works_Experience_Graded.csv", index=False)